In [59]:
import json
import os
import pandas as pd

from dotenv import load_dotenv
from typing import List, Dict, Any
from openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [54]:
pd.set_option('display.width', -1)
pd.set_option('max_colwidth', 1000)

In [56]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("Issue detected with API key")
    
openai = OpenAI()

API key looks good so far


In [57]:
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

### 1. Load JSON data
---

In [31]:
# Load JSON data
with open('./data/convfinqatrain.json', 'r') as f:
    json_data = json.load(f)

### 2. Helper Functions for processing financial documents encoded as JSON file
---

In [29]:
def table_to_string(table):
    """Convert table to string representation."""
    return "\n".join(" | ".join(str(cell) for cell in row) for row in table)

In [48]:
def process_json_entry(entry: Dict[str, Any]) -> Dict[str, Any]:
    """Process a single JSON entry to extract id, merged text, and table string."""
    entry_id = entry.get("id", None)
    parts = []

    # Merge pre_text if available
    if "pre_text" in entry and entry["pre_text"]:
        parts.append(" ".join(entry["pre_text"]))

    # Convert table to a string if it exists
    table_str = table_to_string(entry["table"]) if "table" in entry and entry["table"] else ""
    if table_str:
        parts.append(table_str)

    # Merge post_text if available
    if "post_text" in entry and entry["post_text"]:
        parts.append(" ".join(entry["post_text"]))

    # Merge QA details
    if "qa" in entry and entry["qa"]:
        qa_parts = []
        if "question" in entry["qa"]:
            qa_parts.append(f"Question: {entry['qa']['question']}")
        if "answer" in entry["qa"]:
            qa_parts.append(f"Answer: {entry['qa']['answer']}")
        if "exe_ans" in entry["qa"]:
            qa_parts.append(f"Execution Answer: {entry['qa']['exe_ans']}")
        parts.append("\n".join(qa_parts))
    
    # Extract and Merge dialogue break
    dialogue_break = entry.get("annotation", {}).get("dialogue_break", "")
    if dialogue_break:
        parts.append(f"Dialogue Break: {dialogue_break}")

    # Combine all parts into a single text field
    full_text = "\n\n".join(parts)

    return {"id": entry_id, "text": full_text, "possible_questions":dialogue_break, "table": table_str}    

In [49]:
# Limit to the first ten documents and process them
processed_data = [process_json_entry(entry) for entry in json_data[:10]]

In [50]:
# Convert to a pandas DataFrame
df = pd.DataFrame(processed_data)
df

,id,text,possible_questions,table
0,Single_JKHY/2009/page_28.pdf-3,"26 | 2009 annual report in fiscal 2008 , reven...",[what is the net cash from operating activitie...,2008 | year ended june 30 2009 2008 | year end...
1,Single_RSG/2008/page_114.pdf-2,substantially all of the goodwill and other in...,"[what were revenues in 2008?, what were they i...",| year ended december 31 2008 ( unaudited ) |...
2,Single_AAPL/2002/page_23.pdf-1,in a new business model such as the retail seg...,"[what was the total of net sales in 2001?, and...",| 2002 | 2001 | 2000\nnet sales | $ 5742 | $ ...
3,Single_UPS/2009/page_33.pdf-2,( 1 ) includes shares repurchased through our ...,[what was the change in the performance of the...,| 12/31/04 | 12/31/05 | 12/31/06 | 12/31/07 |...
4,Double_UPS/2009/page_33.pdf,( 1 ) includes shares repurchased through our ...,[what was the fluctuation of the performance p...,| 12/31/04 | 12/31/05 | 12/31/06 | 12/31/07 |...
5,Single_CE/2010/page_134.pdf-2,tax returns for 2001 and beyond are open for e...,[how many shares are subject to outstanding aw...,| shares available for awards | shares subjec...
6,Single_JPM/2013/page_104.pdf-2,management 2019s discussion and analysis 110 j...,[what was the net change in value of litigatio...,as of or for the year ended december 31 ( in m...
7,Double_MAS/2012/page_92.pdf,masco corporation notes to consolidated financ...,[what was the difference in the company 2019s ...,| 2012 | 2011\nbalance at january 1 | $ 102 |...
8,Single_HIG/2004/page_122.pdf-2,the following table identifies the company 201...,[what is the value of obligations due within 1...,| total | less than 1 year | 1-3 years | 3-5 ...
9,Single_SLG/2013/page_133.pdf-4,"during the years ended december 31 , 2013 , 20...","[what was the total, in millions, capitalized ...",| 2013 | 2012 | 2011\nbalance at beginning of...


### 3. Embed Contents of text field using openai
---

In [58]:
# Initialize OpenAI Embeddings model
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

/var/folders/4f/c9ycdnn56lx6fjnv2my0qwk80000gn/T/ipykernel_25293/3483609981.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")


In [60]:
# Generate embeddings for each text field in the DataFrame
df["embedding"] = df["text"].apply(lambda x: embedding_model.embed_query(x) if x else [])

# Display the DataFrame
df

,id,text,possible_questions,table,embedding
0,Single_JKHY/2009/page_28.pdf-3,"26 | 2009 annual report in fiscal 2008 , revenues in the credit union systems and services business segment increased 14% ( 14 % ) from fiscal 2007 . all revenue components within the segment experienced growth during fiscal 2008 . license revenue generated the largest dollar growth in revenue as episys ae , our flagship core processing system aimed at larger credit unions , experienced strong sales throughout the year . support and service revenue , which is the largest component of total revenues for the credit union segment , experienced 34 percent growth in eft support and 10 percent growth in in-house support . gross profit in this business segment increased $ 9344 in fiscal 2008 compared to fiscal 2007 , due primarily to the increase in license revenue , which carries the highest margins . liquidity and capital resources we have historically generated positive cash flow from operations and have generally used funds generated from operations and short-term borrowings on our re...","[what is the net cash from operating activities in 2009?, what about in 2008?, what is the difference?, what percentage change does this represent?]",2008 | year ended june 30 2009 2008 | year ended june 30 2009 2008 | year ended june 30 2009\nnet income | $ 103102 | $ 104222 | $ 104681\nnon-cash expenses | 74397 | 70420 | 56348\nchange in receivables | 21214 | -2913 ( 2913 ) | -28853 ( 28853 )\nchange in deferred revenue | 21943 | 5100 | 24576\nchange in other assets and liabilities | -14068 ( 14068 ) | 4172 | 17495\nnet cash from operating activities | $ 206588 | $ 181001 | $ 174247,"[-0.02153088252851385, 0.034172120073308956, 0.05282134352185816, 0.008284767759422634, 0.0413218943695141, 0.00043241867119711043, -0.01924730388096669, 0.0481182615650618, 0.006341007751049621, -0.0050598935069896085, 0.021843514465834764, -0.037651857723046, -0.0393917284105596, -0.0006983264187464601, 0.027144678717693684, 0.02328434493713962, -0.002162943042281868, -0.0098479321026399, -0.0496950166980687, -0.037869343887291546, 0.03150794529494478, 0.01954634409717543, 0.011146037395073938, 0.004737065916189487, -0.010826608665874397, 0.025581515305798955, -0.06094979624394287, -0.03740718811657611, -0.014190809235577662, -0.03419930724082345, 0.050021240356501795, -0.04151219336624514, 0.04559001144839967, 0.033302186592197226, -0.015115114257750771, 0.015128706910185485, -0.026818453196615518, 0.0162025329717855, 0.0479551460105551, -0.04330643466349759, 0.02355619984847894, 0.00708181149969285, -0.009331408516153227, -0.03686348201918763, -0.06774615971420042, -0.007244924..."
1,Single_RSG/2008/page_114.pdf-2,"substantially all of the goodwill and other intangible assets recorded related to the acquisition of allied are not deductible for tax purposes . pro forma information the consolidated financial statements presented for republic include the operating results of allied from the date of the acquisition . the following pro forma information is presented assuming the merger had been completed as of january 1 , 2007 . the unaudited pro forma information presented below has been prepared for illustrative purposes and is not intended to be indicative of the results of operations that would have actually occurred had the acquisition been consummated at the beginning of the periods presented or of future results of the combined operations ( in millions , except share and per share amounts ) . year ended december 31 , year ended december 31 , ( unaudited ) ( unaudited ) .\n\n | year ended december 31 2008 ( unaudited ) | year ended december 31 2007 ( unaudited )\nrevenue | $ 9362.2 | $ 9244....","[what were revenues in 2008?, what were they in 2007?, what was the net change?, what is the percent change?]",| year ended december 31 2008 ( unaudited ) | year ended december 31 2007 ( unaudited )\nrevenue | $ 9362.2 | $ 9244.9\nincome from continuing operations available to common stock